[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jesherjoshua/seds-spaceweek-2022/blob/mothership/pulsars/pulsars.ipynb)

In [ ]:
import numpy as np
from astropy.io import fits
from time import time
from time import perf_counter
import sys
import os
from timeit import timeit
import random
import matplotlib.pyplot as plt 

# WHAT ARE PULSARS

## ABOUT THE DATASET
The Dataset contains images of a part of the sky where the location of a pulsar is confirmed. The images are in the **fits** (Flexible Image Transport System) format. We will be using the **astropy** package to open and access these fits format images. The Data is collected from the Murchison Widefield Array (**MWA**) in Australia. The Dataset is already pre-processed and is centered to the faintest the quasar signal in said image. 

## MORE ABOUT THE MWA:
The Murchison Widefield Array (MWA) is a low-frequency radio telescope in Western Australia. The front-end of the MWA consists of 4,096 spider-like antennas arranged in 256 regular grids called ‘tiles’, spread over several kilometres within the Murchison Radio-astronomy Observatory (MRO).
The MWA’s particular attributes include:

- a very wide field of view (hundreds of square degrees)
- high angular resolution (several arcminutes)
- wide frequency range (70–300 MHz) with flexible tuning, and
![MWA](./MWA.png)

### LOAD DATA
**lets get a list of the files in the directory**

In [ ]:
samples = sorted(os.listdir('/Users/jesherjoshua/Downloads/pulsars_fits_11/'))
files=sorted(os.listdir('/Users/jesherjoshua/Downloads/pulsars_fits_7330/'))
print("No. of files in fits_11: ",len(samples))
print("No.of files in fits_7330: ",len(files))

In [ ]:
type(fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/0000.fits')[0].data)

In [ ]:
fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/0000.fits')[0].data


**hmm the data is in the form of a numpy array why is that ?**

# WHY NUMPY ?
**lets find out by performing a few basic arithmetic operations**

In [ ]:
lis= random.sample(range(10**5),1000)
np_array=np.array(lis)
print(lis)

## Numpy array vs List (Mean)

In [ ]:
#list - operations


s = "sum(lis)/len(lis)"
timeit(s,setup ="import random; lis= random.sample(range(10**5),1000)",number=1000)

In [ ]:
#using the mean function from the statistics module


s="mean(lis)"
timeit(s,setup ="import random; from statistics import mean; lis= random.sample(range(10**5),1000)",number=1000)


In [ ]:
#np


s="np.divide(np.sum(lis),len(lis))"
timeit(s,setup="import numpy as np; import random;lis= np.array(random.sample(range(10**5),1000))",number=1000)

## Numpy array vs List (Median)

In [ ]:
#list - operations

s = """n=len(lis)
mid=int(n/2)
list=sorted(lis)
if(n%2==0):
    median=(list[mid]+list[mid-1])/2
else:
    median=list[mid]"""

timeit(s,setup="import random;lis= random.sample(range(10**5),1000);",number=1000)

In [ ]:
#using the median method from the statistics module


s="median(lis)"

timeit(s,setup="import random;from statistics import median; lis= random.sample(range(10**5),1000);",number=1000)

In [ ]:
#np
s = """median=np.median(lis)"""
timeit(s,setup="import numpy as np; import random;lis= np.array(random.sample(range(10**5),1000))",number=1000)


# DATA EXPLORATION

## Sample Set

In [ ]:

for i in range(10):
    fh=fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_11/'+samples[i])
    data = fh[0].data
    plt.figure(figsize=(50,50))
    plt.subplot(1,10,i+1)
    #plt.tight_layout()
    plt.imshow(data)
    #plt.title(files[i])

## Actual Dataset

In [ ]:
plt.figure(figsize=(50,50))
for i in range(20):
    fh=fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/'+files[i+100])
    data = fh[0].data
    plt.subplot(4,5,i+1,)
    #plt.tight_layout()
    plt.imshow(data)
    plt.title(files[i])

# RECOGNIZING PULSARS

# MEAN STACKING TO DETECT UNDERLYING PULSARS

## Using Minimal Sample Set

In [ ]:
data=0
for i in range(len(samples)):
    data+=fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_11/'+samples[i])[0].data
avg=data/len(samples)
plt.imshow(avg)

In [ ]:
type(data)

## Using a Larger Dataset

In [ ]:
acc=0
start=time()
for i in range(len(files)):
    acc+=fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/'+files[i])[0].data
mean=acc/len(files)
end=time()-start
print('Time taken: ',end)
plt.imshow(mean)

In [ ]:
acc=0
start=time()
for i in range(len(files)):
    acc+=fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/'+files[i])[0].data
mean=np.divide(acc,len(files))
end=time()-start
print('Time taken: ',end)
plt.imshow(mean)

**Mean Stacking** with the Sample Set reveals the presence of a **Pulsar**. But, doing the same with the entire dataset shows the lack of a **Pulsar**



### **Did the computer make a mistake? Where did the Pulsar go? is there another way?**

# ANALYZING USING MEDIAN STACK

## Using all the data

In [ ]:
l=[]
start=time()
for i in range(len(files)):
    l.append(fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_7330/'+files[i])[0].data)
median_stack=np.dstack(l)
median=np.median(median_stack,axis=2)
end=time()-start
print('Time taken: ',end)
plt.imshow(median)

In [ ]:
median_stack.ndim

## Using the Minimal Sample Set

In [ ]:
l=[]
start=time()
for i in range(len(samples)):
    l.append(fits.open('/Users/jesherjoshua/Downloads/pulsars_fits_11/'+samples[i])[0].data)
median_stack=np.dstack(l)
median=np.median(median_stack,axis=2)
end=time()-start
print('Time taken: ',end)
plt.imshow(median)

# Why does the median perform better than the mean at this task ?

**let's see an example**

In [ ]:
arr=np.array([1,2,3,4,5,10,25,1000])
print("Mean of the array: ",np.mean(arr))
print("Median of the array: ",np.median(arr))

In [ ]:
plt.scatter(arr,arr)
plt.scatter(np.mean(arr),np.mean(arr))
plt.scatter(np.median(arr),np.median(arr))
plt.legend(['Data','Mean','Median'])

We infer from the above experiment:
**The Median is more resistant to outliers rather than the Mean**

In [ ]:
import running_median as rm
import matplotlib.pyplot as plt
plt.imshow(rm.median_approx_fits('./pulsars_fits_11/'+samples))